In [106]:
first = 'https://portal.gmpplus.org/en-US/cdb/cdb-cc-search/?&certstatus=23&scheme=777620000&orderby=name%20DESC'
second = 'https://portal.gmpplus.org/en-US/cdb/cdb-cc-search/?&certstatus=23&scheme=777620000&page=2&orderby=name%20DESC'
detail_page = 'https://portal.gmpplus.org/_portal/modal-form-template-path/2ab10dab-d681-4911-b881-cc99413f07b6?id=fc6ed32c-9b8a-ed11-81ad-000d3a2e4c6e&entityformid=67F1D23C-1A08-E811-811F-5065F38B0311&languagecode=undefined'

In [107]:
import pytz
from datetime import datetime
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from requests.adapters import HTTPAdapter, Retry

requests.adapters.DEFAULT_POOLSIZE = 100
retries = Retry(total=20)
# 세션을 생성하고 ConnectionPool 크기를 설정
session = requests.Session()
session.mount('https://', requests.adapters.HTTPAdapter(pool_connections=100, pool_maxsize=100, max_retries=retries))

In [110]:
def get_url(page_no, type='FSA'):
    if type == 'type':
        if page_no == 1:
            return 'https://portal.gmpplus.org/en-US/cdb/cdb-cc-search/?&certstatus=23&scheme=777620000&orderby=name%20DESC'
        else:
            return f'https://portal.gmpplus.org/en-US/cdb/cdb-cc-search/?&certstatus=23&scheme=777620000&page={page_no}&orderby=name%20DESC'
    else: # FRA
        if page_no == 1:
            return 'https://portal.gmpplus.org/en-US/cdb/cdb-cc-search/?&certstatus=23&scheme=777620001&orderby=name%20DESC'
        else:
            return f'https://portal.gmpplus.org/en-US/cdb/cdb-cc-search/?&certstatus=23&scheme=777620001&page={page_no}&orderby=name%20DESC'
    
    

In [112]:
def get_attr_by_id(obj, id, tag="input", attr="value"):
    item = obj.find(tag, id=id)
    if item:
        return item.get(attr, None)
    return None
    
def get_text_by_id(obj, id, tag="textarea"):
    item = obj.find(tag, id=id)
    if item:
        return item.text.replace('\r\n', '  ').strip()
    return None
    

In [113]:
def item_to_json(item):
    result = {}
    result["company_name"] = get_attr_by_id(item, id="name")
    result["compony_address"] = get_text_by_id(item, id="address1_composite", tag="textarea")
    result["company_phone"] = get_attr_by_id(item, id="telephone1")
    result['company_email'] = get_attr_by_id(item, "emailaddress1")
    result['website'] = get_attr_by_id(item, 'websiteurl')

    result["company_fax"] = get_attr_by_id(item, id="fax")
    result['postal_address'] = get_text_by_id(item, 'address2_composite')
    result["registration_number"] = get_attr_by_id(item, id="accountnumber")
    result["registration_number_old"] = get_attr_by_id(item, id="crmp_chainpointnummer")
    result['vessel_registration_number_eu_number'] = get_attr_by_id(item, id="crmp_eunumber")
    result['legal_business_registration'] = get_attr_by_id(item, "crmp_legalbusinessregistration")
    result['vessel_name'] = get_attr_by_id(item, 'crmp_nameofship')

    return result

In [ ]:
# FSA
import time
combine_results = []
for no in range(1, 201):
    print(no)
    response = session.get(get_url(no))
    result_json = response.json()
    for crmp in result_json['results']:
        detail_res = session.get(f'https://portal.gmpplus.org/_portal/modal-form-template-path/2ab10dab-d681-4911-b881-cc99413f07b6?id={crmp["id"]}&entityformid=67F1D23C-1A08-E811-811F-5065F38B0311&languagecode=undefined')
        item = BeautifulSoup(detail_res.text, 'html.parser')
        result = item_to_json(item)
        combine_results.append(result)
        time.sleep(0.1)


In [115]:
# FRA
import time
combine_results_fra = []
for no in range(1, 43):
    print(no)
    response = session.get(get_url(no, 'FRA'))
    result_json = response.json()
    for crmp in result_json['results']:
        detail_res = session.get(f'https://portal.gmpplus.org/_portal/modal-form-template-path/2ab10dab-d681-4911-b881-cc99413f07b6?id={crmp["id"]}&entityformid=67F1D23C-1A08-E811-811F-5065F38B0311&languagecode=undefined')
        item = BeautifulSoup(detail_res.text, 'html.parser')
        result = item_to_json(item)
        combine_results_fra.append(result)
        time.sleep(0.1)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42


In [105]:
import pandas as pd
df = pd.DataFrame(combine_results)
df.to_csv('./output/gmpplus_fsa_20240116.csv', index=False)

In [116]:
import pandas as pd
df_fra = pd.DataFrame(combine_results_fra)
df_fra.to_csv('./output/gmpplus_fra_20240116.csv', index=False)